# What

This is going to add the content extraction of the reports form atsb and tsbi nto the engine.

It carries on the work from #257 of adding pdf text extraction to the engine.

In [ ]:
import pandas as pd
from plotnine import ggplot, geom_histogram, aes, facet_wrap

import engine.extract.ReportExtracting as ReportExtracting

import importlib
import os
import shutil

importlib.reload(ReportExtracting)

Here I will setup some of the global values needed for this workbook.

In [ ]:
output_folder = '../../output'
report_text = pd.read_pickle(os.path.join(output_folder, 'parsed_reports.pkl'))
report_text['year'] = report_text['report_id'].str.extract('(\d{4})')
report_text

## ATSB

In [ ]:
importlib.reload(ReportExtracting)
atsb_reports = report_text[report_text['report_id'].str.contains('ATSB')]
atsb_reports['content_section'] = atsb_reports.apply(
    lambda x: ReportExtracting.ReportExtractor(x['text'], x['report_id']).extract_contents_section(), axis=1
)
atsb_reports

In [ ]:
print(atsb_reports['content_section'].notna().value_counts())

failed = atsb_reports[atsb_reports['content_section'].isnull()]
(
    ggplot(atsb_reports.assign(has_content_section=atsb_reports['content_section'].notna()), aes(x='year'))
    + geom_histogram()
    + facet_wrap('has_content_section', ncol=1)
)

## TSB

TSB is currently running into the issue that it doesn't have content sections for mosts of its reports. I will do ATSB first and will hopefully have a more robust content section extractor. Otherwise I will look into having itgenerate a content sections based on regex searches of the report.

In [ ]:
tsb_reports = report_text[report_text['report_id'].str.startswith('TSB')]

importlib.reload(ReportExtracting)
tsb_reports['content_section'] = tsb_reports.apply(
    lambda x: ReportExtracting.ReportExtractor(x['text'], x['report_id']).extract_contents_section(), axis=1
)
tsb_reports

In [ ]:
print(tsb_reports['content_section'].notna().value_counts())
# Make histogram of years with two colours for content section exist and not
failed = tsb_reports[tsb_reports['content_section'].isnull()]
(
    ggplot(tsb_reports.assign(has_content_section=tsb_reports['content_section'].notna()), aes(x='year'))
    + geom_histogram()
    + facet_wrap('has_content_section', ncol=1)
)


## TAIC

Even though it is already supported I should know the full information

In [ ]:
taic_reports = report_text[report_text['report_id'].str.startswith('TAIC')]

importlib.reload(ReportExtracting)
taic_reports['content_section'] = taic_reports.apply(
    lambda x: ReportExtracting.ReportExtractor(x['text'], x['report_id']).extract_contents_section(), axis=1
)
taic_reports

In [ ]:
print(taic_reports['content_section'].notna().value_counts())

failed = taic_reports[taic_reports['content_section'].isnull()]

(
    ggplot(taic_reports.assign(has_content_section=taic_reports['content_section'].notna()), aes(x='year'))
    + geom_histogram()
    + facet_wrap('has_content_section', ncol=1)
)

## Checking the output

At first test there is about 10% failed for TAIC, 50% for ATSB and 16% for TSB.
That is just using the regular expression as it currently is.

We can see that TAIC is only failing once it gets back before 2005 so that can be ignored for now. However both TSB and ATSB have a lot of missing content sections.

In [ ]:
# Save to text files parsed files to let me inspect them
shutil.rmtree('parsed_reports/', ignore_errors=True)

os.makedirs('parsed_reports/', exist_ok=True)
for index, text, content_section in failed[['report_id', 'text', 'content_section']].sample(10, random_state=42).to_records(index=False):
    with open(os.path.join('parsed_reports', f'{index}.txt'), 'w') as f:
        f.write(text)
    with open(os.path.join('parsed_reports', f'{index}_content_section.txt'), 'w') as f:
        f.write(content_section if content_section is not None else '')
    shutil.copy(os.path.join(output_folder, 'report_pdfs', f"{index}.pdf"), os.path.join('parsed_reports', f"{index}.pdf"))